## Bin Packing

We have a bunch of items of different sizes and we want to put them in bins. How should we pack them to Minimize the number of bins needed?

In [1]:
from pulp import *
import numpy as np

### 1. First lets make some fake data

In [2]:
items=['item_%d'%i for i in range(50)]

In [3]:
item_sizes = dict( (i,np.random.randint(1,20)) for i in items)

### The Model

Lets model the each possible bins as having at most N spots to be filled by item 0,...,N.

$$x_{i,b} = \begin{cases}
    1, & \text{if item i is in bin b } \\
    0, & \text{otherwise}
\end{cases}
$$

We need to make sure each item is placed in exactly one bin. Ie for any given item, summing $x_{i,b}$ along the bins should equal 1.

$$\sum_{b} x_{i,b} = 1 \ \forall i$$

We also need to make sure that if a bin is used, it is not used beyond its capacity. 

$$\sum_{i} x_{i,b} \leq \text{bin_capacity}*y_{b} \ \forall b$$

Finally, we are trying to minimize the number of needed bins, so our objective is:

$$\text{Minimize} \ \sum_{b} y_{b}$$

In [4]:
bin_size = 40

In [5]:
bins = ['bin_%d'%i for i in range(len(items))]

In [6]:
x = LpVariable.dicts('x',[(i,b) for i in items for b in bins],0,1,LpBinary)

In [7]:
y = LpVariable.dicts('bin',bins,0,10, LpBinary)

In [8]:
#create the problme
prob=LpProblem("bin_packing",LpMinimize)

In [9]:
#the objective
cost = lpSum([ y[b] for b in bins])
prob+=cost

In [10]:
#every item is placed in exactly one bin
for i in items:
    prob+= lpSum([x[i,b] for b in bins]) == 1

In [11]:
#if a bin is used, it has a capacity constraint
for b in bins:
    prob+=lpSum([ item_sizes[i]*x[i,b] for i in items]) <= bin_size*y[b]

### Solve it!

In [12]:
%time prob.solve()
print(LpStatus[prob.status])

CPU times: user 65.5 ms, sys: 7.36 ms, total: 72.9 ms
Wall time: 1.18 s
Optimal


And the result:

In [13]:
print(value(prob.objective))

14.0


In [14]:
for b in bins:
    if value(y[b]) !=0:
        print(b,':',', '.join([ i for i in items if value(x[i,b]) !=0 ]))

bin_0 : item_3, item_4, item_5, item_19
bin_1 : item_6, item_40, item_42, item_48
bin_7 : item_31, item_44
bin_10 : item_8, item_13, item_28, item_30
bin_11 : item_23, item_27, item_32
bin_12 : item_35, item_37, item_45
bin_13 : item_2, item_16, item_17, item_18, item_25
bin_14 : item_21, item_41, item_43
bin_15 : item_9, item_22, item_24
bin_16 : item_10, item_11, item_12
bin_17 : item_1, item_14, item_33, item_38, item_46
bin_18 : item_0, item_20, item_29
bin_19 : item_15, item_26, item_47, item_49
bin_23 : item_7, item_34, item_36, item_39


In [15]:
print(value(prob.objective))

14.0
